In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minRR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxRR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["RR"].mean()
    or_std  = d["RR"].std(ddof=1)
    df_out.loc[idx, "RR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minRR"] = d["2.5%"].min()
    df_out.loc[idx, "maxRR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
data = pd.read_csv('data/BBAG-long.csv')

In [4]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


## Odd ratios with macrosocials (new vars)

### Without Iteration

In [5]:
results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP'
    
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:


        c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
        c_results_merge_df_all.reset_index(drop = True, inplace = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [6]:
for covar in covar_list:
    c_dfRR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_new-covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfRR)

GDP_per_capita


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
0,1.217742,1.307343,1.261747,12.836560,1.023213e-37,Mono,GDP_per_capita
1,0.835581,0.918387,0.876006,-5.491802,3.978537e-08,One,GDP_per_capita
2,0.781228,0.837831,0.809035,-11.875469,1.587413e-32,Two,GDP_per_capita
3,0.819299,0.918154,0.867319,-4.898322,9.665871e-07,Three,GDP_per_capita
4,0.764308,0.820651,0.791978,-12.853218,8.250566e-38,Total,GDP_per_capita


GDP


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
5,1.434527,1.534246,1.483549,23.007191,3.949366e-117,Mono,GDP
6,0.891888,0.979211,0.934530,-2.841613,4.488590e-03,One,GDP
7,0.718794,0.774481,0.746119,-15.385472,2.048761e-53,Two,GDP
8,0.597610,0.664725,0.630275,-17.000421,8.153193e-65,Three,GDP
9,0.651147,0.696477,0.673431,-23.028697,2.405126e-117,Total,GDP


### 1000-Iteration

In [7]:

results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP' 
    
)


n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
            c_results_merge_df_all.reset_index(drop = True, inplace = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["RR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)



In [8]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [9]:
for covar in covar_list:
    c_dfRR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_new-covar-' + covar + '-iter.xlsx')

    print(covar)
    display(c_dfRR)

GDP_per_capita


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
0,GDP_per_capita,Mono,0.286518,0.613504,0.996295,1.252563,1.251511,1.253615,1.211075,1.294732,10.0
1,GDP_per_capita,One,0.142282,0.544040,0.995612,0.871323,0.869966,0.872680,0.832842,0.910840,10.0
2,GDP_per_capita,Two,0.273007,0.401946,0.995194,0.817632,0.817062,0.818202,0.791823,0.844812,10.0
3,GDP_per_capita,Three,0.333219,0.612461,0.997541,0.875952,0.874575,0.877328,0.830285,0.923974,10.0
4,GDP_per_capita,Total,0.285414,0.613139,0.996323,0.797802,0.797132,0.798473,0.771778,0.825183,10.0


GDP


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
5,GDP,Mono,0.123297,0.436031,0.995325,1.473368,1.471836,1.474901,1.427658,1.521877,10.0
6,GDP,One,0.220829,0.491304,0.987590,0.926443,0.925131,0.927755,0.886398,0.968498,10.0
7,GDP,Two,0.176444,0.235910,0.985425,0.756500,0.755985,0.757015,0.731041,0.783079,10.0
8,GDP,Three,0.176630,0.495974,0.992617,0.637840,0.636651,0.639030,0.605887,0.670647,10.0
9,GDP,Total,0.117313,0.429470,0.994621,0.678091,0.677387,0.678796,0.656450,0.699833,10.0
